In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from mtcnn import MTCNN

# Set the dataset path and target size for FaceNet
dataset_path = "dataset_directory"
target_size = (160, 160)  # FaceNet expects 160x160 pixel input

# Initialize the MTCNN detector for face alignment
detector = MTCNN()

def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error reading {image_path}")
        return None
    
    # Detect faces
    result = detector.detect_faces(image)
    
    if result:
        # Assume the first face detected is the target face
        face = result[0]
        x, y, width, height = face['box']
        x, y = max(0, x), max(0, y)

        # Crop the face region
        face_image = image[y:y+height, x:x+width]
        
        # Resize to target size (160x160)
        face_image = cv2.resize(face_image, target_size)
        
        # Normalize pixel values to [0, 1]
        face_image = face_image / 255.0
        
        # Convert to RGB format for FaceNet
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
        
        return face_image
    else:
        print(f"No face detected in {image_path}")
        return None

def preprocess_dataset(dataset_path):
    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        
        # Only proceed if it's a directory
        if not os.path.isdir(person_path):
            continue

        print(f"Processing images for {person_name}")
        
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            
            # Preprocess the image
            processed_image = preprocess_image(img_path)
            
            if processed_image is not None:
                # Save preprocessed image as a numpy array for later use
                save_path = os.path.join(person_path, f"processed_{img_name}")
                np.save(save_path, processed_image)
                print(f"Saved processed image at {save_path}")
            else:
                print(f"Skipping {img_path}")

# Run the preprocessing on the entire dataset
preprocess_dataset(dataset_path)

Processing images for rahul


error: OpenCV(4.10.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x46dff480::Set<3,4,-1>,struct cv::impl::A0x46dff480::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)
